In [37]:
from pathlib import Path
from langchain.document_loaders import Docx2txtLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv
import chromadb
from langchain.text_splitter import RecursiveCharacterTextSplitter #recommended text splitter by langchain

In [12]:
load_dotenv()  

True

# Reduced data

 Trying with first a very reduced version of 2015. 
This version has an example of text that does not exist in 2023 (1-Click) and text that does (Family Sharing). 
Aim is to protoype the process and prompting on this small set before extending to the entire dataset


# Load documents

In [56]:
REDUCED_DATA_DIR = Path("data", "reduced")
toc_2015_fname = Path(REDUCED_DATA_DIR, "Jan 2015_reduced.docx")
toc_2023_fname = Path(REDUCED_DATA_DIR, "Mar 2023.docx")

In [57]:
loader_2015 = Docx2txtLoader(str(toc_2015_fname)) # str reqd for loader
data_2015 = loader_2015.load()
data_2015

[Document(page_content="TERMS AND CONDITIONS\n\nA. ITUNES STORE, MAC APP STORE, APP STORE, AND IBOOKS STORE TERMS OF SALE\nB. ITUNES STORE TERMS AND CONDITIONS\nC. MAC APP STORE, APP STORE AND IBOOKS STORE TERMS AND CONDITIONS\n\n\n1-Click®\n\n1-Click is a registered service mark of Amazon.com, Inc., used under license. 1-Click is a convenient feature that allows you to make a purchase from the Stores with a single click of your mouse or other input device. When accessing the Stores on your computer, 1-Click purchasing may be activated via the dialog that appears when you click a Buy button. (You may reset this selection at any time by clicking Reset Warnings in your Account information). When accessing the Stores on your Apple-branded products running iOS such as an iPad, iPod touch, or iPhone (“iOS Device”), 1-Click is activated for each transaction by tapping the button showing the price of the product, which reveals the Buy button. When 1-Click is activated, clicking or tapping the

In [58]:
loader_2023 = Docx2txtLoader(str(toc_2023_fname))
data_2023 = loader_2023.load()
data_2023

[Document(page_content='Apple Media Services Terms and Conditions\n\nThese terms and conditions create a contract between you and Apple (the “Agreement”). Please read the Agreement carefully.\n\nA. INTRODUCTION TO OUR SERVICES\n\nThis Agreement governs your use of Apple’s Services (“Services” – e.g., and where available, App Store, Apple Arcade, Apple Books, Apple Fitness+, Apple Music, Apple News, Apple News+, Apple One, Apple Podcasts, Apple Podcasts Subscriptions, Apple TV, Apple TV+, Apple TV Channels, Game Center, iTunes), through which you can buy, get, license, rent or subscribe to content, Apps (as defined below), and other in-app services (collectively, “Content”). Content may be offered through the Services by Apple or a third party. Our Services are available for your use in your country or territory of residence (“Home Country”). By creating an account for use of the Services in a particular country or territory you are specifying it as your Home Country. To use our Service

# create embeddings

In [62]:
"""
Info on chunk size and chunk overlap here
https://github.com/langchain-ai/langchain/issues/2026

GPT4 turbo has a token length of 4k, and since I dont need many return tokens, I can 
set the chunk size to quite large, 
"""
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap  = 20,
)

In [68]:
texts_2015 = text_splitter.split_documents(data_2015)
texts_2023 = text_splitter.split_documents(data_2023)

for idx, text in enumerate(texts_2015):
    texts_2015[idx].metadata['year'] = '2015'
for idx, text in enumerate(texts_2023):
    texts_2023[idx].metadata['year'] = '2023'

In [61]:
len(texts_2015), len(texts_2023)

(6, 33)

In [71]:
embeddings = OpenAIEmbeddings()
client_2015 = Chroma.from_documents(
    documents = texts_2015, 
    embedding = embeddings, 
    collection_name="toc_collection",
    persist_directory="chroma_db", 
)
embeddings = OpenAIEmbeddings()
client_2023 = Chroma.from_documents(
    documents = texts_2023, 
    embedding = embeddings, 
    collection_name="toc_collection",
    persist_directory="chroma_db", 
)
